In [ ]:
from pyoptools.all import *
from math import sqrt, pi, radians,cos, sin
import random
import pandas as pd


lens_radius = 25
microlens_radius = 2.7
microlens_f = 0.2
CCD = CCD(size=(30, 30))
L1 = SphericalLens(radius=lens_radius, curvature_s1=1./100., curvature_s2=-1./100., thickness=10, material=material.schott["N-BK7"])

#microlens = SphericalLens(radius=microlens_radius, curvature_s1=1./100., curvature_s2=-1./100., thickness=1.2, material=material.schott["N-BK7"])
MShape = Circular(radius=microlens_radius)
microlens = IdealLens(shape=MShape, f=microlens_f)

num_xmicrolens = 5
num_ymicrolens = 5
microlens_pitch = microlens_radius * 2
array_z_distance = 95.09
CCD_z = 100


microlens_locations = []
for i in range(num_xmicrolens):
    for j in range(num_ymicrolens):
        x_pos = i * microlens_pitch 
        y_pos = j * microlens_pitch
        
        microlens_location = (x_pos - 10, y_pos - 10, array_z_distance)
        microlens_locations.append(microlens_location)


lens_location = (0, 0, 0)
CCD_location = (0, 0, CCD_z)

complist = [(L1, lens_location, (0, 0, 0)), (CCD, CCD_location, (0, 0, 0))]
#OA = Ray(pos=(0,0,0), dir=(0,0,1), intensity=100, wavelength=.350)

for m in microlens_locations:
    complist.append((microlens, m, (0, 0, 0)))
    
S = System(complist=complist, n=1)
    
num_ps = 100
source_wavelength = 0.550
num_rays = 2

source_z_distance = -275

R = [] 
for i in range(num_ps):
    #print(i)
    if i < num_ps // 2:
        x = random.uniform(-lens_radius, 0)
        y = 0
    else:
        x = 0
        y = random.uniform(-lens_radius, 0)
    
    pos = (x, y, source_z_distance)
    
    for j in range(num_rays):

        phi = random.uniform(0, 2 * pi)
        
        theta_deg = random.uniform(0,15)
        theta = radians(theta_deg)
        
        dir_z = cos(theta) 
        dir_x = sin(theta)*cos(phi)
        dir_y = sin(theta)*sin(phi)
    
        ray = Ray(pos=pos, dir=(dir_x, dir_y, dir_z), wavelength=source_wavelength)
        R.append(ray)

#f = microlens.paraxial_constants()
#print(f)        
        
S.ray_add(R)
#S.ray_add(OA)
S.propagate()

spot_diagram_c(CCD)
Plot3D(S, center=(0, 0, 200), size=(450, 200), scale=2, rot=[(0, -pi/2, 0), (pi/20, -pi/10, 0)])
